In [4]:
import numpy as np
import matplotlib.pyplot as plt

from numba import njit


In [5]:
class Fields():
    
    def __init__(self, l, N = 10):
        # Domain is a cube with N^3 points
        self.l               = l
        self.xmin, self.xmax = l/2, l/2
        self.ymin, self.ymax = l/2, l/2
        self.zmin, self.zmax = l/2, l/2
        self.h               = l / N
        
        self.A   = np.zeros((N,N,N))
        self.B   = np.zeros((N,N,N))
        self.DA  = np.zeros((N,N,N))
        self.DB  = np.zeros((N,N,N))
        self.KA  = np.zeros((N,N,N))
        self.KB  = np.zeros((N,N,N))
        self.Dal = np.zeros((N,N,N))
        self.al  = np.zeros((N,N,N))

# RHS of eqs (3)
These are decomposed using finite differences.

In [6]:
@njit        
def to_coord(i,j,k,h,l):
    x = - l/2 + i * h
    y = - l/2 + j * h
    z = - l/2 + k * h
    return x,y,z

@njit
def angs(i,j,k,h,l):
    x,y,z = to_coord(i,j,k,h,l)
    return np.arccos(z / np.sqrt(x**2 + y**2 + z**2)), np.arctan2(y / x)

@njit
def der_r(f,i,j,k,h,l):
    """
    TO DO:: Pass only the cube that contains the data needed, not everything
    
    f      :: field to derive wrt the radial direction
    i,j,k  :: position in the grid
    h      :: spacing of the grid
    l      :: length of the domain (1D)
    """
    th, ph = angs(i,j,k,h,l)

    out1 = np.cos(ph) * np.sin(th) * (f[i+1,j,k] - f[i-1,j,k]) ## x derivative
    out2 = np.sin(ph) * np.sin(th) * (f[i,j+1,k] - f[i,j-1,k]) ## y derivative
    out3 =              np.cos(th) * (f[i,j,k+1] - f[i,j,k-1]) ## z derivative
    
    return (out1 + out2 + out3) * 0.5 / h

In [9]:
def ev_A(field, i,j,k):
    """
    field :: class object containing all the fields (self in Fields class)
    i,j,k   :: position in the grid
    """
    return - 2 * field.al[i,j,k] * field.A[i,j,k] * field.KA[i,j,k]

def ev_B(field, i,j,k):
    """
    field :: class object containing all the fields (self in Fields class)
    i,j,k   :: position in the grid
    """
    return - 2 * field.al[i,j,k] * field.B[i,j,k] * field.KB[i,j,k]

def ev_DA(field, i,j,k):
    """
    field :: class object containing all the fields (self in Fields class)
    i,j,k   :: position in the grid
    """
    p1 = field.KA[i,j,k] * field.Dal[i,j,k]
    p2 = der_r(field.KA, i,j,k, field.h, field.l)
    return - 2 * field.al[i,j,k] * (p1 + p2)

def ev_DB(field, i,j,k):
    """
    field :: class object containing all the fields (self in Fields class)
    i,j,k   :: position in the grid
    """
    p1 = field.KB[i,j,k] * field.Dal[i,j,k]
    p2 = der_r(field.KB, i,j,k, field.h, field.l)
    return - 2 * field.al[i,j,k] * (p1 + p2)


def ev_KA(field, i,j,k):
    """
    field :: class object containing all the fields (self in Fields class)
    i,j,k   :: position in the grid
    """
    x,y,z = to_coord(i,j,k, field.h, field.l)
    r = np.sqrt(x**2 + y**2 + z**2)
    
    p1 = der_r(field.Dal + field.DB, i,j,k, field.h, field.l)
    p2 = field.Dal[i,j,k]**2 + 0.5 * (field.Dal[i,j,k] * field.DA[i,j,k] + field.DB[i,j,k]**2 - field.DA[i,j,k] * field.DB[i,j,k])
    p3 = - field.A[i,j,k] * field.KA[i,j,k] * (field.KA[i,j,k] + 2*field.KB[i,j,k])
    p4 = - (field.DA[i,j,k] - 2 * field.DB[i,j,k]) / r
    return - field.al[i,j,k] * (p1 + p2 + p3 + p4) / field.A[i,j,k]
    
def ev_KB(field, i,j,k):
    """
    field :: class object containing all the fields (self in Fields class)
    i,j,k   :: position in the grid
    """
    x,y,z = to_coord(i,j,k, field.h, field.l)
    r = np.sqrt(x**2 + y**2 + z**2)
    
    p1 = der_r(field.DB, i,j,k, field.h, field.l)
    p2 = field.Dal[i,j,k] * field.DB[i,j,k] + field.DB[i,j,k]**2 - 0.5 * field.DA[i,j,k] * field.DB[i,j,k]
    p3 = - (field.DA[i,j,k] - 2 * field.Dal[i,j,k] - 4 * field.DB[i,j,k]) / r
    p4 = - 2 * (field.A[i,j,k] - field.B[i,j,k]) / (field.B[i,j,k] * r**2)
    p5 = field.al[i,j,k] * field.KB[i,j,k] * (field.KA[i,j,k] + 2*field.KB[i,j,k])
    return - 0.5 * field.al[i,j,k] * (p1+p2+p3+p4) / field.A[i,j,k] + p5
    